In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Read Parquet Data") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .getOrCreate()
print("done")

done


In [3]:
# read parquet file
df = spark.read.parquet("hdfs://namenode:9000/user/hive/warehouse/web_logs_partitioned")

# show schema
df.printSchema()

root
 |-- client: string (nullable = true)
 |-- datetime: string (nullable = true)
 |-- request: string (nullable = true)
 |-- status: string (nullable = true)
 |-- size: string (nullable = true)
 |-- referrer: string (nullable = true)
 |-- useragent: string (nullable = true)
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)



In [4]:

# show data frame
df.show()

+-------------+--------------------+--------------------+------+-----+--------------------+--------------------+----------+----+-----+---+
|       client|            datetime|             request|status| size|            referrer|           useragent|      date|year|month|day|
+-------------+--------------------+--------------------+------+-----+--------------------+--------------------+----------+----+-----+---+
| 54.36.149.41|22/Jan/2019:03:56...|GET /filter/27|13...|   200|30577|                   -|Mozilla/5.0 (comp...|2019-01-22|2019|    1| 22|
|  31.56.96.51|22/Jan/2019:03:56...|GET /image/60844/...|   200| 5667|https://www.zanbi...|Mozilla/5.0 (Linu...|2019-01-22|2019|    1| 22|
|  31.56.96.51|22/Jan/2019:03:56...|GET /image/61474/...|   200| 5379|https://www.zanbi...|Mozilla/5.0 (Linu...|2019-01-22|2019|    1| 22|
|40.77.167.129|22/Jan/2019:03:56...|GET /image/14925/...|   200| 1696|                   -|Mozilla/5.0 (comp...|2019-01-22|2019|    1| 22|
|  91.99.72.15|22/Jan/2019:

In [6]:
df_partition = spark.read.parquet(
    "hdfs://namenode:9000/user/hive/warehouse/web_logs_partitioned/year=2019/month=1/day=22"
)
df_partition.show()


+-------------+--------------------+--------------------+------+-----+--------------------+--------------------+----------+
|       client|            datetime|             request|status| size|            referrer|           useragent|      date|
+-------------+--------------------+--------------------+------+-----+--------------------+--------------------+----------+
| 54.36.149.41|22/Jan/2019:03:56...|GET /filter/27|13...|   200|30577|                   -|Mozilla/5.0 (comp...|2019-01-22|
|  31.56.96.51|22/Jan/2019:03:56...|GET /image/60844/...|   200| 5667|https://www.zanbi...|Mozilla/5.0 (Linu...|2019-01-22|
|  31.56.96.51|22/Jan/2019:03:56...|GET /image/61474/...|   200| 5379|https://www.zanbi...|Mozilla/5.0 (Linu...|2019-01-22|
|40.77.167.129|22/Jan/2019:03:56...|GET /image/14925/...|   200| 1696|                   -|Mozilla/5.0 (comp...|2019-01-22|
|  91.99.72.15|22/Jan/2019:03:56...|GET /product/3189...|   200|41483|                   -|Mozilla/5.0 (Wind...|2019-01-22|
|40.77.1

In [ ]:
streaming_df = spark.readStream \
    .format("parquet") \
    .option("path", "hdfs://namenode:9000/user/hive/warehouse/web_logs_partitioned") \
    .load()
print("done")

In [ ]:
query = streaming_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

In [ ]:
query.awaitTermination()